In [1]:
#AI學習規劃秘書（Reflection模式）
#安裝套件
!pip install aisuite[all] gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3

In [3]:
#匯入模組
import os
import aisuite as ai
import gradio as gr
from google.colab import userdata

#設定API金鑰與模型
import os

# 直接設定 Groq API 金鑰（勿公開分享）
api_key = "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"
os.environ['GROQ_API_KEY'] = api_key

provider_writer = "groq"
model_writer = "llama-3.3-70b-versatile"

provider_reviewer = "groq"
model_reviewer = "llama-3.3-70b-versatile"

#定義AI回應函式
def reply(system="請使用台灣常用語回應", prompt="hi", provider="groq", model="llama3-70b-8192"):
    client = ai.Client()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=f"{provider}:{model}", messages=messages
    )
    return response.choices[0].message.content

#定義角色人設
system_writer = """
請使用台灣中文。
你是一位熱情又專業的學習顧問，會根據使用者輸入的目標與背景，提出具體的2個月學習建議。
請輸出：1.總學習目標拆解（4週為單位） 2.每週任務簡述。
風格親切、鼓舞人心，文字控制在300字以內。
"""

system_reviewer = """
請使用台灣中文。
你是一位理性審查員，會針對學習計劃內容給予具體改進建議。
如果任務過多、安排不合理或目標模糊，請提出批評並給予調整建議，字數控制在150字以內。
"""

#Reflection模式主函式
def reflect_plan(user_prompt):
    # Step 1: 撰寫者初步規劃
    plan = reply(system_writer, user_prompt,
                 provider=provider_writer,
                 model=model_writer)

    # Step 2: 評估者審查意見
    review = reply(system_reviewer, plan,
                   provider=provider_reviewer,
                   model=model_reviewer)

    # Step 3: 撰寫者根據建議微調內容
    second_prompt = f"""
以下是我原本的規劃：\n{plan}\n\n
這是你的審查建議：\n{review}\n\n
請根據審查內容，重新整理你的學習建議，保持語氣親切，限制在300字內。
"""
    refined = reply(system_writer, second_prompt,
                    provider=provider_writer,
                    model=model_writer)
    return plan, review, refined

In [ ]:
#Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 AI 學習規劃秘書：Reflection 模式")
    user_input = gr.Textbox(label="請描述你的學習目標、背景、時間，讓AI幫你規劃！")
    btn = gr.Button("生成規劃")

    with gr.Row():
        out1 = gr.Textbox(label="📋 初步建議")
        out2 = gr.Textbox(label="🔍 審查建議")
        out3 = gr.Textbox(label="🧾 優化後建議")

    btn.click(reflect_plan, inputs=[user_input], outputs=[out1, out2, out3])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1c4e5f95dea446e8b7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
